In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import json, io, base64, re, os, zipfile
import plotly.graph_objects as go
from IPython.display import display, HTML, clear_output
import plotly.express as px
import ipywidgets as widgets

# Initialize settings
import _settings as settings

# Global variables from settings
plotly_colors = settings.plotly_colors

In [2]:
class DataTransformation:
    def __init__(self):
        self.merged_df = None
        self.output_area = None
        self.merged_uploader = None

    def create_download_link(self, file_path, label):
        """Create a download link for a file."""
        if os.path.exists(file_path):
            # Read file content and encode it as base64
            with open(file_path, 'rb') as f:
                content = f.read()
            b64_content = base64.b64encode(content).decode('utf-8')

            # Generate the download link HTML
            return widgets.HTML(f"""
                <a download="{os.path.basename(file_path)}" 
                   href="data:application/octet-stream;base64,{b64_content}" 
                   style="color: #0366d6; text-decoration: none; margin-left: 20px; font-size: 14px;">
                    {label}
                </a>
            """)
        else:
            # Show an error message if the file does not exist
            return widgets.HTML(f"""
                <span style="color: red; margin-left: 20px; font-size: 14px;">
                    File "{file_path}" not found!
                </span>
            """)

    def setup_data_loading_ui(self):
        """Initialize and display the data loading UI."""
        # Create file upload widget
        self.merged_uploader = widgets.FileUpload(
            accept='.csv,.txt,.tsv,.xlsx',
            multiple=False,
            description='Upload Data File',
            layout=widgets.Layout(width='300px'),
            style={'description_width': 'initial'}
        )

        self.output_area = widgets.Output()

        # Create upload box with example link
        merged_box = widgets.HBox([
            self.merged_uploader,
            self.create_download_link("example_merged_dataframe.csv", "Example")
        ], layout=widgets.Layout(align_items='center'))

        # Create left column with upload widgets
        upload_widgets = widgets.VBox([
            widgets.HTML("<h4>Upload Data File:</h4>"),
            merged_box,
            self.output_area
        ], layout=widgets.Layout(
            width='300px',
            margin='0 20px 0 0'
        ))

        # Create container for status display
        self.status_area = widgets.Output(
            layout=widgets.Layout(
                width='300px',
                margin='0 0 0 20px'
            )
        )

        display(upload_widgets,
                self.status_area)

        # Register observer
        self.merged_uploader.observe(self._on_merged_upload_change, names='value')

    def _validate_and_clean_data(self, df):
        """
        Validate and clean the uploaded data, dropping rows with blank values in key columns.
        Returns tuple of (cleaned_df, warnings, errors)
        """
        warnings = []
        errors = []

        # Check required columns exist
        required_columns = [
            'Master Protein Accessions', 
            'Positions in Proteins',
            'unique ID'
        ]
        
        # Check that at least one Avg_ column exists
        avg_columns = [col for col in df.columns if col.startswith('Avg_')]
        if not avg_columns:
            errors.append("No columns starting with 'Avg_' found in the data")
            return None, warnings, errors
            
        # Add Avg_ columns to required columns
        required_columns.extend(avg_columns)
        
        missing = set(required_columns) - set(df.columns)
        if missing:
            errors.append(f"Missing required columns: {', '.join(missing)}")
            return None, warnings, errors

        cleaned_df = df.copy()

        # Handle blank values by dropping rows and issuing warnings
        for column in required_columns:
            blank_count = cleaned_df[column].isna().sum()
            if blank_count > 0:
                warnings.append(f"Dropping {blank_count} rows with blank values in {column} column")
                cleaned_df = cleaned_df.dropna(subset=[column])

        # Check for invalid characters in non-blank rows
        if len(cleaned_df) > 0:
            # Check Positions in Proteins
            invalid_pos = cleaned_df['Positions in Proteins'].apply(
                lambda x: ',' in str(x) or ':' in str(x)
            )
            if invalid_pos.any():
                errors.append(
                    "Found invalid characters (',' or ':') in Positions in Proteins column. "
                    "Please update the file and upload again."
                )

            # Check Master Protein Accessions
            invalid_acc = cleaned_df['Master Protein Accessions'].apply(
                lambda x: ',' in str(x) or ':' in str(x)
            )
            if invalid_acc.any():
                errors.append(
                    "Found invalid characters (',' or ':') in Master Protein Accessions column. "
                    "Please update the file and upload again."
                )

        return cleaned_df, warnings, errors

    def _load_merged_data(self, file_data):
        """
        Load and validate merged data file
        Returns tuple of (dataframe, status)
        """
        try:
            content = bytes(file_data.content)
            filename = file_data.name
            extension = filename.split('.')[-1].lower()

            file_stream = io.BytesIO(content)

            # Load data based on file extension
            try:
                if extension == 'csv':
                    df = pd.read_csv(file_stream)
                elif extension in ['txt', 'tsv']:
                    df = pd.read_csv(file_stream, delimiter='\t')
                elif extension == 'xlsx':
                    df = pd.read_excel(file_stream)
                else:
                    display(HTML(f'<b style="color:red;">Error: Unsupported file format</b>'))
                    return None, 'no'
            except Exception as e:
                display(HTML(f'<b style="color:red;">Error reading file: {str(e)}</b>'))
                return None, 'no'

            # Validate and clean data
            cleaned_df, warnings, errors = self._validate_and_clean_data(df)

            # Display warnings about dropped rows
            if warnings:
                warning_html = "<br>".join([
                    f'<b style="color:orange;">Warning: {w}</b>'
                    for w in warnings
                ])
                display(HTML(warning_html))

            # Display errors if any
            if errors:
                error_html = "<br>".join([
                    f'<b style="color:red;">Error: {e}</b>'
                    for e in errors
                ])
                display(HTML(error_html))
                return None, 'no'

            if cleaned_df is not None and len(cleaned_df) > 0:
                # Process protein information
                
                # Add information about remaining rows and processed proteins
                display(HTML(
                    f'<b style="color:green;">Processed data contains {len(cleaned_df)} rows '
                    f'after removing blank values.</b><br>'
                ))
                return cleaned_df, 'yes'
            else:
                display(HTML('<b style="color:red;">Error: No valid data rows remaining after cleaning</b>'))
                return None, 'no'

        except Exception as e:
            display(HTML(f'<b style="color:red;">Error processing file: {str(e)}</b>'))
            return None, 'no'

    def _on_merged_upload_change(self, change):
        """Handle merged data file upload"""
        if change['type'] == 'change' and change['name'] == 'value':
            with self.output_area:
                self.output_area.clear_output()
                if change['new'] and len(change['new']) > 0:
                    file_data = change['new'][0]
                    df, status = self._load_merged_data(file_data)
                    if status == 'yes' and df is not None:
                        self.merged_df = df  # Only set merged_df if validation passed
                        display(HTML(
                            f'<b style="color:green;">Data imported successfully with '
                            f'{df.shape[0]} rows and {df.shape[1]} columns.</b>'
                        ))

In [3]:
class BioactivePlotter:
    def __init__(self, data_transformer):
        self.data_transformer = data_transformer
        self.plot_output = widgets.Output()
        self.export_output = widgets.Output()
        self.current_fig_abs = None
        self.current_fig_rel = None
        self.info_output = widgets.Output()
        self.current_pie_charts = []

        # Define the function list excluding "Minor Functions (<1%)"
        self.function_list = [
            'ACE-inhibitory', 'Ameliorates insulin resistance', 'Antianxiety', 'Anticancer',
            'Antimicrobial', 'Antioxidant', 'Antithrombotic', 'Cholesterol regulation',
            'Cytotoxic', 'DPP-IV Inhibitory', 'Immunomodulatory', 'Increase calcium uptake',
            'Increase cellular growth', 'Opioid', 'Osteoanabolic', 'Prolyl endopeptidase-inhibitory',
            'Antithrombitic', 'Increase mucin secretion', 'Satiety', 'Cytomodulatory'
        ]
        
        # Initialize widgets
        self.setup_widgets()
     
    def setup_widgets(self):
        """Initialize UI widgets"""
        # Create plot control widgets
        self.plot_button = widgets.Button(
            description='Generate/Update Data',
            button_style='success',
            icon='refresh',
            layout=widgets.Layout(width='200px')
        )
        
        self.export_button = widgets.Button(
            description='Export Data to Excel',
            button_style='info',
            icon='download',
            layout=widgets.Layout(width='200px'),
            disabled=True
        )
        
        self.download_plot_button = widgets.Button(
            description='Download Interactive Plot',
            button_style='info',
            icon='file',
            layout=widgets.Layout(width='200px'),
            disabled=True
        )

        """self.download_all_png_button = widgets.Button(
            description='Download All Plots as PNG',
            button_style='info',
            icon='archive',
            layout=widgets.Layout(width='200px'),
            disabled=True
        )"""
        
        # Add group selection widget
        self.group_select = widgets.SelectMultiple(
            description='Groups:',
            options=[],
            layout=widgets.Layout(width='300px', height='100px'),
            style={'description_width': 'initial'}
        )
        
        # Update plot type selection to remove 'All Plots'
        self.plot_type = widgets.RadioButtons(
            options=['Stacked Bar Plots', 'Grouped Bar Plots', 'Pie Charts'],
            value='Stacked Bar Plots',
            description='Plot Type:',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='300px')
        )
        
        # Add bar plot type selection
        self.bar_plot_type = widgets.RadioButtons(
            options=['Absolute Absorbance', 'Relative Absorbance', 'Absolute Count', 'Relative Count'],
            value='Absolute Absorbance', 
            description=f'Bar Plot Type:',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='300px')
        )
        
        # Add color scheme selector
        self.color_scheme = widgets.Dropdown(
            options=plotly_colors,
            value='HSV',
            description='All Plots Color Scheme:',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='300px')
        )
        
        # Add label customization
        self.xlabel_widget = widgets.Text(
            description='Bar Plot X Label:',
            placeholder='Enter x-axis label',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='300px')
        )
        
        self.ylabel_widget = widgets.Text(
            description='Bar Plot Y Label:',
            placeholder='Enter y-axis label',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='300px')
        )
        
        self.legend_widget = widgets.Text(
            description='Bar Plot Legend Title',
            placeholder='Enter a custom legend title',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='300px')
        )
        
        self.title_widget = widgets.Text(
            description='Bar Plot Plot Title',
            placeholder='Enter a custom plot title',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='300px')
        )
        # Create layout with sections
        radio_box = widgets.HBox([
            self.plot_type, self.bar_plot_type],
        layout=widgets.Layout(width='400px'))
        
        # Create layout with sections
        controls_box = widgets.VBox([
            widgets.HTML("<h4>Plot Controls:</h4>"),
            self.group_select,
            radio_box
        ])
        
        appearance_box = widgets.VBox([
            widgets.HTML("<h4>Appearance Settings:</h4>"),
            self.xlabel_widget,
            self.ylabel_widget,
            self.legend_widget,
            self.title_widget,
            self.color_scheme
        ])
        
        button_box = widgets.VBox([
            widgets.HTML("<h4>Actions:</h4>"),
            widgets.VBox([
                self.plot_button, 
                self.export_button, 
                self.download_plot_button,
                #self.download_all_png_button
            ])
        ])
        
        # Create main layout
        self.widget_box = widgets.VBox([
            controls_box, 
            appearance_box,
            button_box,
            self.info_output,
            self.plot_output,
            self.export_output
        ])
        
        # Add button callbacks
        self.plot_button.on_click(self._on_plot_button_click)
        self.export_button.on_click(self._on_export_button_click)
        self.download_plot_button.on_click(self._on_download_plot_click)
        #self.download_all_png_button.on_click(self._on_download_all_png_click)
        
        # Add observer for data changes
        self.data_transformer.merged_uploader.observe(self._update_group_options, names='value')

        # Add observer for plot type changes to show/hide bar plot type selector
        self.plot_type.observe(self._on_plot_type_change, names='value')

    def _update_group_options(self, change):
        """Update group selection options when data changes"""
        if self.data_transformer.merged_df is not None:
            # Get all Avg_ columns
            avg_columns = [col.replace('Avg_', '') for col in self.data_transformer.merged_df.columns 
                         if col.startswith('Avg_')]
            
            # Update group selection options
            self.group_select.options = avg_columns
            # Select all groups by default
            self.group_select.value = avg_columns
      
      
    def _on_plot_type_change(self, change):
        """Show/hide bar plot type selector based on plot type"""
        if change['new'] in ['Stacked Bar Plots', 'Grouped Bar Plots']:
            self.bar_plot_type.layout.display = ''
        else:
            self.bar_plot_type.layout.display = 'none'

    def _process_bioactive_data(self):
        """Process bioactive peptide data for visualization"""
        if self.data_transformer.merged_df is None:
            return None
            
        df = self.data_transformer.merged_df
        if 'function' not in df.columns:
            return None
            
        unique_function_absorbance = {}
        avg_columns = [col for col in df.columns if col.startswith('Avg_')]
        
        for column in avg_columns:
            grouping_variable = column.replace('Avg_', '')
            
            # Filter and process data
            temp_df = df[['unique ID', 'function', column]].copy()
            temp_df = temp_df[
                (temp_df[column] != 0) & 
                temp_df[column].notna() &
                temp_df['function'].notna()
            ]
            
            if temp_df.empty:
                continue
            
            # Process functions
            temp_df.loc[:, 'function'] = temp_df['function'].fillna('').str.split(';')
            exploded_df = temp_df.explode('function')
            exploded_df.loc[:, 'function'] = exploded_df['function'].str.strip()
            exploded_df = exploded_df[exploded_df['function'] != '']
            
            if not exploded_df.empty:
                function_grouped = exploded_df.groupby('function')[column].sum()
                unique_function_absorbance[grouping_variable] = function_grouped.to_dict()
        
        return unique_function_absorbance
        
    def process_function_percentages(self, data_dict, threshold=1):
        """Process data to combine functions below threshold into 'Minor Functions'"""
        total = sum(data_dict.values())
        processed_data = {}
        minor_functions_sum = 0
        
        # Calculate percentages and filter
        for func, value in data_dict.items():
            percentage = (value / total) * 100
            if percentage >= threshold:
                processed_data[func] = value
            else:
                minor_functions_sum += value
                
        # Add minor functions if any exist
        if minor_functions_sum > 0:
            processed_data[f'Minor Functions (<{threshold}%)'] = minor_functions_sum
            
        return processed_data
        
    def create_pie_charts(self, unique_function_absorbance):
        """Create pie charts for counts and abundances"""
        if not unique_function_absorbance:
            return None
            
        # Process the data to get counts
        results = self._process_export_data()
        if results is None:
            return None
            
        combined_df, combined_count_df, combined_absorbance_df = results
            
        pie_figs = []
        for group in unique_function_absorbance.keys():
            # Get data for this group
            absorbance_data = unique_function_absorbance[group]
            
            # Process abundance data with threshold
            processed_abundance = self.process_function_percentages(absorbance_data, threshold=1)
            
            # Get correct count data from combined_count_df
            count_data = {}
            for func in combined_count_df.index:
                if func != 'Counts of peptides':
                    count = combined_count_df.loc[func, group]
                    if count > 0:
                        count_data[func] = count
                        
            # Process count data with threshold
            processed_counts = self.process_function_percentages(count_data, threshold=1)
            
            # Create count pie chart first
            count_fig = go.Figure()
            count_labels = list(processed_counts.keys())
            count_values = list(processed_counts.values())
            count_colors = self._get_color_sequence(len(count_labels))
            
            count_fig.add_trace(go.Pie(
                labels=count_labels,
                values=count_values,
                marker=dict(colors=count_colors),
                textfont=dict(size=14, color="black"),  # Make labels black
                textinfo='percent+label',
                hovertemplate="Function: %{label}<br>" +
                            "Count: %{value}<br>" +
                            "Percentage: %{percent}<br>" +
                            "<extra></extra>",
                #title=f"Count Distribution<br>{group}"
            ))
            
            # Create abundance pie chart
            abundance_fig = go.Figure()
            labels = list(processed_abundance.keys())
            values = list(processed_abundance.values())
            colors = self._get_color_sequence(len(labels))
            
            abundance_fig.add_trace(go.Pie(
                labels=labels,
                values=values,
                marker=dict(colors=colors),
                textfont=dict(size=14, color="black"),  # Make labels black
                textinfo='percent+label',
                hovertemplate="Function: %{label}<br>" +
                            "Abundance: %{value:.2e}<br>" +
                            "Percentage: %{percent}<br>" +
                            "<extra></extra>",
                #title=f"Abundance Distribution<br>{group}<br><br>"
            ))
            
            pie_figs.extend([count_fig, abundance_fig])
            
        return pie_figs
        
    def _get_color_sequence(self, n_colors):
        """Get color sequence based on selected scheme"""
        if self.color_scheme.value.lower() in plotly_colors:
            return [f'hsl({h},70%,60%)' for h in np.linspace(0, 330, n_colors)]
        
        try:
            color_sequence = getattr(px.colors.sequential, self.color_scheme.value, None)
            if color_sequence is None:
                color_sequence = getattr(px.colors.diverging, self.color_scheme.value, None)
            
            if color_sequence:
                if n_colors >= len(color_sequence):
                    indices = np.linspace(0, len(color_sequence)-1, n_colors)
                    return [color_sequence[int(i)] for i in indices]
                else:
                    return color_sequence[:n_colors]
        except:
            pass
            
        return [f'hsl({h},70%,60%)' for h in np.linspace(0, 330, n_colors)]

        

    def plot_stacked_bioactive_peptides(self, unique_function_absorbance):
        """Generate interactive Plotly stacked bar plots for bioactive peptides"""
        if not unique_function_absorbance:
            return None, None
            
        # Get all unique functions present in the data
        all_functions = set()
        for group_data in unique_function_absorbance.values():
            all_functions.update(group_data.keys())
            
        # Filter and sort functions
        functions = [f for f in self.function_list if f in all_functions]
        additional_functions = sorted([f for f in all_functions if f not in self.function_list])
        functions.extend(additional_functions)
        
        # Prepare data
        groups = list(unique_function_absorbance.keys())
        colors = self._get_color_sequence(len(functions))
        
        # Get count data if needed
        count_data = None
        if 'Count' in self.bar_plot_type.value:
            results = self._process_export_data()
            if results is not None:
                _, combined_count_df, _ = results
                count_data = combined_count_df

        # Create figures based on plot type
        if self.bar_plot_type.value == 'Absolute Absorbance':
            fig1, fig2 = self._create_abundance_plots(functions, groups, unique_function_absorbance, colors)
        elif self.bar_plot_type.value == 'Relative Absorbance':
            fig1, fig2 = self._create_relative_abundance_plots(functions, groups, unique_function_absorbance, colors)
        elif self.bar_plot_type.value == 'Absolute Count' and count_data is not None:
            fig1, fig2 = self._create_count_plots(functions, groups, count_data, colors)
        elif self.bar_plot_type.value == 'Relative Count' and count_data is not None:
            fig1, fig2 = self._create_relative_count_plots(functions, groups, count_data, colors)
        else:
            return None, None

        return fig1, fig2

    def _create_abundance_plots(self, functions, groups, data, colors):
        """Create absolute abundance plots"""
        fig = go.Figure()
        total_abundances = []
        
        # Calculate abundances
        plot_data = {func: [] for func in functions}
        for group in groups:
            total = 0
            for func in functions:
                abundance = data[group].get(func, 0)
                plot_data[func].append(abundance)
                total += abundance
            total_abundances.append(total)
            
        # Add traces
        for idx, func in enumerate(functions):
            hover_text = [
                f"Function: {func}<br>" +
                f"Sample: {group}<br>" +
                f"Absolute Abundance: {abundance:.2e}"
                for group, abundance in zip(groups, plot_data[func])
            ]
            
            fig.add_trace(go.Bar(
                name=func,
                x=groups,
                y=plot_data[func],
                marker_color=colors[idx],
                hovertext=hover_text,
                hoverinfo='text'
            ))
            
        # Update layout
        fig.update_layout(
            title={
                'text': self.title_widget.value or 'Distribution of Bioactive Peptides by Function',
                'y': 0.95,
                'x': 0.5,
                'xanchor': 'center',
                'yanchor': 'top',
                'font': {'size': 15, 'color': 'black'}
            },
            xaxis_title=self.xlabel_widget.value or 'Sample Type',
            yaxis_title=self.ylabel_widget.value or 'Scaled Absolute Absorbance',
            legend_title=self.legend_widget.value or "Bioactive Function:",
            legend={'yanchor': "top", 'y': 0.95, 'xanchor': "left", 'x': 1.05, 'traceorder': 'normal', 'font': {'size': 12, 'color': 'black'}},
            showlegend=True,
            template='plotly_white',
            height=700,
            width=1000,
            margin=dict(t=100, l=100, r=200),
            hoverlabel=dict(
                bgcolor="white",
                font_size=12,
                font_family="Arial"
            ),
            barmode='stack',
            xaxis=dict(
                showline=True,
                linewidth=1,
                linecolor='black',
                mirror=False
            ),
            yaxis=dict(
                showline=True,
                linewidth=1,
                linecolor='black',
                mirror=False
            )
        )
        fig.update_xaxes(
            tickangle=45,
            title_font={"size": 14},
            tickfont={"size": 12},
            tickfont_color="black",  # Black tick labels
            title_font_color="black",  # Black axis title                
        )
        
        fig.update_yaxes(
            title_font={"size": 14},
            tickfont={"size": 12},
            tickfont_color="black",  # Black tick labels
            title_font_color="black",  # Black axis title
            gridcolor="lightgray",  # Light gray grid lines
            showgrid=True,  # Show grid lines
            zeroline=False,  # Hide zero line
            exponentformat='E',
            showexponent='all'
        )
        
        return fig, None

    def _create_relative_abundance_plots(self, functions, groups, data, colors):
        """Create relative abundance plots"""
        fig = go.Figure()
        
        # Calculate abundances and totals
        plot_data = {func: [] for func in functions}
        total_abundances = []
        
        for group in groups:
            total = 0
            for func in functions:
                abundance = data[group].get(func, 0)
                plot_data[func].append(abundance)
                total += abundance
            total_abundances.append(total)
            
        # Add traces
        for idx, func in enumerate(functions):
            relative_values = [
                100 * abundance / total if total > 0 else 0
                for abundance, total in zip(plot_data[func], total_abundances)
            ]
            
            hover_text = [
                f"Function: {func}<br>" +
                f"Sample: {group}<br>" +
                f"Relative Abundance: {value:.1f}%<br>" +
                f"(Abundance: {abundance:.2e})"
                for group, value, abundance in zip(groups, relative_values, plot_data[func])
            ]
            
            fig.add_trace(go.Bar(
                name=func,
                x=groups,
                y=relative_values,
                marker_color=colors[idx],
                hovertext=hover_text,
                hoverinfo='text'
            ))
            
        # Update layout
        fig.update_layout(
            title={
                'text': self.title_widget.value or 'Relative Distribution of Bioactive Peptides by Function',
                'y': 0.95,
                'x': 0.5,
                'xanchor': 'center',
                'yanchor': 'top',
                'font': {'size': 15, 'color': 'black'}  # Black title text
            },
            xaxis_title=self.xlabel_widget.value or 'Sample Type',
            yaxis_title=self.ylabel_widget.value or 'Relative Abundance (%)',
            legend_title=self.legend_widget.value or "Bioactive Function:",
            legend={'yanchor': "top", 'y': 0.95, 'xanchor': "left", 'x': 1.05, 'traceorder': 'normal', 'font': {'size': 12, 'color': 'black'}},
            showlegend=True,
            template='plotly_white',
            height=700,
            width=1000,
            margin=dict(t=100, l=100, r=200),
            hoverlabel=dict(
                bgcolor="white",
                font_size=12,
                font_family="Arial"
            ),
            barmode='stack',
            
            xaxis=dict(
                showline=True,
                linewidth=1,
                linecolor='black',
                mirror=False
            ),
            yaxis=dict(
                showline=True,
                linewidth=1,
                linecolor='black',
                mirror=False
            )
        )
        fig.update_xaxes(
            tickangle=45,
            title_font={"size": 14},
            tickfont={"size": 12},
            tickfont_color="black",  # Black tick labels
            title_font_color="black",  # Black axis title                
        )
        
        fig.update_yaxes(
            title_font={"size": 14},
            tickfont={"size": 12},
            tickfont_color="black",  # Black tick labels
            title_font_color="black",  # Black axis title
            gridcolor="lightgray",  # Light gray grid lines
            showgrid=True,  # Show grid lines
            zeroline=False,  # Hide zero line
            range=[0, 100]  # Set range for percentage
        )
        
        return fig, None

    def _create_count_plots(self, functions, groups, count_data, colors):
        """Create absolute count plots"""
        fig = go.Figure()
        
        for idx, func in enumerate(functions):
            if func in count_data.index:
                counts = [count_data.loc[func, group] for group in groups]
                
                hover_text = [
                    f"Function: {func}<br>" +
                    f"Sample: {group}<br>" +
                    f"Count: {int(count)}"
                    for group, count in zip(groups, counts)
                ]
                
                fig.add_trace(go.Bar(
                    name=func,
                    x=groups,
                    y=counts,
                    marker_color=colors[idx],
                    hovertext=hover_text,
                    hoverinfo='text'
                ))
                
        # Update layout
        fig.update_layout(
            title={
                'text': self.title_widget.value or 'Distribution of Bioactive Peptide Counts by Function',
                'y': 0.95,
                'x': 0.5,
                'xanchor': 'center',
                'yanchor': 'top',
                'font': {'size': 15, 'color': 'black'}  # Black title text
            },
            xaxis_title=self.xlabel_widget.value or 'Sample Type',
            yaxis_title=self.ylabel_widget.value or 'Peptide Count',
            legend_title=self.legend_widget.value or "Bioactive Function:",
            legend={'yanchor': "top", 'y': 0.95, 'xanchor': "left", 'x': 1.05, 'traceorder': 'normal', 'font': {'size': 12, 'color': 'black'}},
            showlegend=True,
            template='plotly_white',
            height=700,
            width=1000,
            margin=dict(t=100, l=100, r=200),
            hoverlabel=dict(
                bgcolor="white",
                font_size=12,
                font_family="Arial"
            ),
            barmode='stack',
            xaxis=dict(
                showline=True,
                linewidth=1,
                linecolor='black',
                mirror=False
            ),
            yaxis=dict(
                showline=True,
                linewidth=1,
                linecolor='black',
                mirror=False
            )
        )
        fig.update_xaxes(
            tickangle=45,
            title_font={"size": 14},
            tickfont={"size": 12},
            tickfont_color="black",  # Black tick labels
            title_font_color="black",  # Black axis title                
        )
        
        fig.update_yaxes(
            title_font={"size": 14},
            tickfont={"size": 12},
            tickfont_color="black",  # Black tick labels
            title_font_color="black",  # Black axis title
            gridcolor="lightgray",  # Light gray grid lines
            showgrid=True,  # Show grid lines
            zeroline=False,  # Hide zero line
            type='linear',  # Linear scale for counts
            tickformat=",d"  # Format with commas for thousands
        )
        
        return fig, None

    def _create_relative_count_plots(self, functions, groups, count_data, colors):
        """Create relative count plots"""
        fig = go.Figure()
        
        # Calculate total counts for each group
        total_counts = {group: 0 for group in groups}
        for func in functions:
            if func in count_data.index:
                for group in groups:
                    total_counts[group] += count_data.loc[func, group]
        
        for idx, func in enumerate(functions):
            if func in count_data.index:
                relative_counts = []
                for group in groups:
                    count = count_data.loc[func, group]
                    total = total_counts[group]
                    relative = (count / total * 100) if total > 0 else 0
                    relative_counts.append(relative)
                
                hover_text = [
                    f"Function: {func}<br>" +
                    f"Sample: {group}<br>" +
                    f"Relative Count: {rel:.1f}%<br>" +
                    f"(Count: {int(count_data.loc[func, group])})"
                    for group, rel in zip(groups, relative_counts)
                ]
                
                fig.add_trace(go.Bar(
                    name=func,
                    x=groups,
                    y=relative_counts,
                    marker_color=colors[idx],
                    hovertext=hover_text,
                    hoverinfo='text'
                ))
                
        # Update layout
        fig.update_layout(
            title={
                'text': self.title_widget.value or 'Relative Distribution of Bioactive Peptide Counts by Function',
                'y': 0.95,
                'x': 0.5,
                'xanchor': 'center',
                'yanchor': 'top',
                'font': {'size': 15, 'color': 'black'}  # Black title text
            },
            xaxis_title=self.xlabel_widget.value or 'Sample Type',
            yaxis_title=self.ylabel_widget.value or 'Relative Count (%)',
            legend_title=self.legend_widget.value or "Bioactive Function:",
            legend={'yanchor': "top", 'y': 0.95, 'xanchor': "left", 'x': 1.05, 'traceorder': 'normal', 'font': {'size': 12, 'color': 'black'}},
            showlegend=True,
            template='plotly_white',
            height=700,
            width=1000,
            margin=dict(t=100, l=100, r=200),
            hoverlabel=dict(
                bgcolor="white",
                font_size=12,
                font_family="Arial"
            ),
            barmode='stack',
            xaxis=dict(
                showline=True,
                linewidth=1,
                linecolor='black',
                mirror=False
            ),
            yaxis=dict(
                showline=True,
                linewidth=1,
                linecolor='black',
                mirror=False
            )
        )
        fig.update_xaxes(
            tickangle=45,
            title_font={"size": 14},
            tickfont={"size": 12},
            tickfont_color="black",  # Black tick labels
            title_font_color="black",  # Black axis title                
        )
        
        fig.update_yaxes(
            title_font={"size": 14},
            tickfont={"size": 12},
            tickfont_color="black",  # Black tick labels
            title_font_color="black",  # Black axis title
            gridcolor="lightgray",  # Light gray grid lines
            showgrid=True,  # Show grid lines
            zeroline=False,  # Hide zero line
        )
        
        return fig, None
  
    def plot_grouped_bioactive_peptides(self, unique_function_absorbance):
        """Generate interactive Plotly grouped bar plots for bioactive peptides"""
        if not unique_function_absorbance:
            return None, None
            
        # Get all unique functions present in the data
        all_functions = set()
        for group_data in unique_function_absorbance.values():
            all_functions.update(group_data.keys())
            
        # Filter functions to only include those in our predefined list that are also in the data
        functions = [f for f in self.function_list if f in all_functions]
        additional_functions = sorted([f for f in all_functions if f not in self.function_list])
        functions.extend(additional_functions)
        
        # Prepare data
        groups = list(unique_function_absorbance.keys())
        colors = self._get_color_sequence(len(functions))
        
        # Get count data if needed
        count_data = None
        if 'Count' in self.bar_plot_type.value:
            results = self._process_export_data()
            if results is not None:
                _, combined_count_df, _ = results
                count_data = combined_count_df
        
        # Create figure
        fig1 = go.Figure()
        fig2 = None
        
        # Calculate bar positions
        n_functions = len(functions)
        bar_width = 0.8 / n_functions  # Adjust total width of group
        
        # Calculate total values for relative plots
        total_values = {}
        if self.bar_plot_type.value in ['Relative Absorbance', 'Relative Count']:
            for group in groups:
                if self.bar_plot_type.value == 'Relative Count' and count_data is not None:
                    total_values[group] = sum(count_data.loc[f, group] if f in count_data.index else 0 
                                            for f in functions)
                else:
                    total_values[group] = sum(unique_function_absorbance[group].values())
        
        for idx, func in enumerate(functions):
            # Calculate x positions for this function's bars
            x_positions = [i + (idx - n_functions/2 + 0.5) * bar_width for i in range(len(groups))]
            
            if self.bar_plot_type.value == 'Absolute Count' and count_data is not None:
                # Get count values
                values = [count_data.loc[func, group] if func in count_data.index else 0 for group in groups]
                hover_text = [
                    f"Function: {func}<br>" +
                    f"Sample: {group}<br>" +
                    f"Count: {int(value)}"
                    for group, value in zip(groups, values)
                ]
            elif self.bar_plot_type.value == 'Relative Count' and count_data is not None:
                # Calculate relative counts
                values = []
                for group in groups:
                    count = count_data.loc[func, group] if func in count_data.index else 0
                    total = total_values[group]
                    relative = (count / total * 100) if total > 0 else 0
                    values.append(relative)
                
                hover_text = [
                    f"Function: {func}<br>" +
                    f"Sample: {group}<br>" +
                    f"Relative Count: {value:.1f}%<br>" +
                    f"(Count: {int(count_data.loc[func, group] if func in count_data.index else 0)})"
                    for group, value in zip(groups, values)
                ]
            elif self.bar_plot_type.value == 'Absolute Absorbance':
                # Get abundance values
                values = [unique_function_absorbance[group].get(func, 0) for group in groups]
                hover_text = [
                    f"Function: {func}<br>" +
                    f"Sample: {group}<br>" +
                    f"Absolute Abundance: {value:.2e}"
                    for group, value in zip(groups, values)
                ]
            else:  # Relative Abundance
                # Calculate relative abundances
                values = []
                for group in groups:
                    abundance = unique_function_absorbance[group].get(func, 0)
                    total = total_values[group]
                    relative = (abundance / total * 100) if total > 0 else 0
                    values.append(relative)
                
                hover_text = [
                    f"Function: {func}<br>" +
                    f"Sample: {group}<br>" +
                    f"Relative Abundance: {value:.1f}%<br>" +
                    f"(Abundance: {unique_function_absorbance[group].get(func, 0):.2e})"
                    for group, value in zip(groups, values)
                ]
            
            fig1.add_trace(go.Bar(
                name=func,
                x=x_positions,
                y=values,
                width=bar_width * 0.9,  # Slight gap between bars
                marker_color=colors[idx],
                hovertext=hover_text,
                hoverinfo='text'
            ))
        
        # Update layout
        title = {
            'Absolute Absorbance': 'Distribution of Bioactive Peptides by Function',
            'Relative Absorbance': 'Relative Distribution of Bioactive Peptides by Function',
            'Absolute Count': 'Distribution of Bioactive Peptide Counts by Function',
            'Relative Count': 'Relative Distribution of Bioactive Peptide Counts by Function'
        }[self.bar_plot_type.value]
        
        yaxis_title = {
            'Absolute Absorbance': 'Scaled Absolute Absorbance',
            'Relative Absorbance': 'Relative Abundance (%)',
            'Absolute Count': 'Peptide Count',
            'Relative Count': 'Relative Count (%)'
        }[self.bar_plot_type.value]
        
        fig1.update_layout(
            title={
                'text': self.title_widget.value or title,
                'y': 0.95,
                'x': 0.5,
                'xanchor': 'center',
                'yanchor': 'top',
                'font': {'size': 15, 'color': 'black'}
            },
            xaxis_title=self.xlabel_widget.value or 'Sample Type',
            yaxis_title=self.ylabel_widget.value or yaxis_title,
            legend_title=self.legend_widget.value or "Bioactive Function:",
            legend={'yanchor': "top", 'y': 0.95, 'xanchor': "left", 'x': 1.05, 'traceorder': 'normal', 'font': {'size': 12, 'color': 'black'}},
            showlegend=True,
            template='plotly_white',
            height=700,
            width=1000,
            margin=dict(t=100, l=100, r=200),
            hoverlabel=dict(
                bgcolor="white",
                font_size=12,
                font_family="Arial"
            ),
            barmode='group',
            xaxis=dict(
                showline=True,
                linewidth=1,
                linecolor='black',
                mirror=False
            ),
            yaxis=dict(
                showline=True,
                linewidth=1,
                linecolor='black',
                mirror=False
            )
        )
        # Update axis properties
        fig1.update_xaxes(
            ticktext=groups,
            tickvals=list(range(len(groups))),
            tickangle=45,
            title_font={"size": 14},
            tickfont={"size": 12},
            tickfont_color="black",  # Black tick labels
            title_font_color="black",  # Black axis title     
        )
        
        # Set y-axis format based on plot type
        if self.bar_plot_type.value == 'Absolute Absorbance':
            fig1.update_yaxes(
                type='log',
                exponentformat='E',
                showexponent='all',
                 title_font={"size": 14},
                tickfont={"size": 12},
                tickfont_color="black",  # Black tick labels
                title_font_color="black",  # Black axis title
                gridcolor="lightgray",  # Light gray grid lines
                showgrid=True,  # Show grid lines
                zeroline=False,  # Hide zero line

            )
        elif self.bar_plot_type.value == 'Absolute Count':
            fig1.update_yaxes(
                type='linear',
                tickformat=",d",  # Format with commas for thousands
                title_font={"size": 14},
                tickfont={"size": 12},
                tickfont_color="black",  # Black tick labels
                title_font_color="black",  # Black axis title
                gridcolor="lightgray",  # Light gray grid lines
                showgrid=True,  # Show grid lines
                zeroline=False,  # Hide zero line
                
            )
        else:  # Relative Abundance or Relative Count
            fig1.update_yaxes(
                type='linear',
                range=[0, 100],
                title_font={"size": 14},
                tickfont={"size": 12},
                tickfont_color="black",  # Black tick labels
                title_font_color="black",  # Black axis title
                gridcolor="lightgray",  # Light gray grid lines
                showgrid=True,  # Show grid lines
                zeroline=False,  # Hide zero line
                
            )
            
        return fig1, fig2
    def _on_plot_button_click(self, b):
        """Handle plot button click"""
        if not self.group_select.value:
            with self.info_output:
                self.info_output.clear_output(wait=True)
                display(HTML("<b style='color:red'>Please select at least one group.</b>"))
            return
            
        with self.plot_output:
            self.plot_output.clear_output(wait=True)
            
            # Process and plot data
            unique_function_absorbance = self._process_bioactive_data()
            if unique_function_absorbance:
                # Filter data for selected groups
                selected_groups = list(self.group_select.value)
                filtered_absorbance = {k: v for k, v in unique_function_absorbance.items() 
                                    if k in selected_groups}
                
                if self.plot_type.value in ['Stacked Bar Plots', 'Grouped Bar Plots']:
                    if self.plot_type.value == 'Stacked Bar Plots':
                        self.current_fig_abs, self.current_fig_rel = self.plot_stacked_bioactive_peptides(
                            filtered_absorbance
                        )
                    else:
                        self.current_fig_abs, self.current_fig_rel = self.plot_grouped_bioactive_peptides(
                            filtered_absorbance
                        )
                        
                    if self.current_fig_abs is not None:
                        self.current_fig_abs.show()
                        
                elif self.plot_type.value == 'Pie Charts':
                    # Create pie charts
                    self.current_pie_charts = []
                    pie_figs = self.create_pie_charts(filtered_absorbance)
                    if pie_figs:
                        for i in range(0, len(pie_figs), 2):
                            if i + 1 < len(pie_figs):
                                fig = go.Figure()
                                
                                # Add the first pie chart (Count)
                                for trace in pie_figs[i].data:
                                    trace.domain = {'row': 0, 'column': 0, 'x': [0.02, 0.38]} 
                                    fig.add_trace(trace)
                                
                                # Add the second pie chart (Abundance)
                                for trace in pie_figs[i+1].data:
                                    trace.domain = {'row': 0, 'column': 1, 'x': [0.62, 0.98]}  
                                    fig.add_trace(trace)
                                
                                group_name = selected_groups[i//2]
                                fig.update_layout(
                                    height=700,
                                    width=1600,
                                    margin=dict(t=150, b=50, l=100, r=100),
                                    annotations=[
                                        dict(
                                            text=f"Count Distribution<br>{group_name}",
                                            x=0.20,
                                            y=1.1,
                                            font=dict(size=20, color='black'),
                                            showarrow=False,
                                            xanchor='center',
                                            yanchor='bottom'
                                        ),
                                        dict(
                                            text=f"Abundance Distribution<br>{group_name}",
                                            x=0.8,
                                            y=1.1,
                                            font=dict(size=20, color='black'),
                                            showarrow=False,
                                            xanchor='center',
                                            yanchor='bottom'
                                        )
                                    ],
                                    showlegend=False
                                )
                                
                                # Update traces with valid attributes
                                fig.update_traces(
                                    textfont_size=14,          # Set text font size for the traces
                                    textposition='outside',    # Position text outside for pie charts
                                    pull=0.02                  # Slightly separate slices for better spacing (if applicable for pie)
                                )
                                
                                # Update layout for axis title and tick colors
                                fig.update_layout(
                                    title_font_color="black",   # Set title font color to black
                                    xaxis=dict(
                                        tickfont=dict(color="black"),  # Set x-axis tick labels to black
                                    ),
                                    yaxis=dict(
                                        tickfont=dict(color="black"),  # Set y-axis tick labels to black
                                    )
                                )
                                self.current_pie_charts.append(fig)
                                fig.show()
                
                # Enable the export and download buttons
                self.export_button.disabled = False
                self.download_plot_button.disabled = False
                #self.download_all_png_button.disabled = False
            else:
                print("No bioactive data available for plotting.")
                self.export_button.disabled = True
                self.download_plot_button.disabled = True
                #self.download_all_png_button.disabled = True
                
    def _process_export_data(self):
        """Process data for export into Excel format"""
        unique_function_absorbance = self._process_bioactive_data()
        if not unique_function_absorbance:
            return None
            
        # Get all groups and functions
        groups = list(unique_function_absorbance.keys())
        all_functions = set()
        for group_data in unique_function_absorbance.values():
            all_functions.update(group_data.keys())
            
        # Calculate function counts
        df = self.data_transformer.merged_df
        summed_function_count = {}
        unique_function_counts = {}
        unique_function_count_averages = {}
        summed_function_abundance = {}
        
        for group in groups:
            abundance_column = f'Avg_{group}'
            if abundance_column not in df.columns:
                continue
                
            # Filter and process data
            temp_df = df[['unique ID', 'function', abundance_column]].copy()
            temp_df = temp_df[
                (temp_df[abundance_column] != 0) & 
                temp_df[abundance_column].notna() &
                temp_df['function'].notna()
            ]
            
            # Drop duplicates and calculate counts
            filtered_df = temp_df.drop_duplicates(subset='unique ID')
            unique_peptide_count = filtered_df['unique ID'].nunique()
            total_sum = filtered_df[abundance_column].sum()
            
            # Store the totals
            summed_function_abundance[group] = total_sum
            summed_function_count[group] = unique_peptide_count
            
            # Process functions
            filtered_df.loc[:, 'function'] = filtered_df['function'].fillna('').str.split(';')
            exploded_df = filtered_df.explode('function')
            exploded_df.loc[:, 'function'] = exploded_df['function'].str.strip()
            exploded_df = exploded_df[exploded_df['function'] != '']
            
            if not exploded_df.empty:
                # Count functions
                function_counts = exploded_df['function'].value_counts().to_dict()
                unique_function_counts[group] = function_counts
                
                # Calculate averages (using 1 since we're using averaged columns)
                function_averages = {func: count for func, count in function_counts.items()}
                unique_function_count_averages[group] = function_averages
        
        # Create DataFrames for export
        peptide_count_df = pd.DataFrame.from_dict(
            summed_function_count,
            orient='index',
            columns=['Counts of peptides']
        )
        
        function_count_df = pd.DataFrame.from_dict(
            unique_function_counts,
            orient='index'
        ).fillna(0).astype(int)
        
        combined_count_df = pd.concat([peptide_count_df, function_count_df], axis=1).T
        
        # Create abundance DataFrames
        peptide_absorbance_df = pd.DataFrame.from_dict(
            summed_function_abundance,
            orient='index',
            columns=['Summed Absorbance']
        )
        
        function_absorbance_df = pd.DataFrame.from_dict(
            unique_function_absorbance,
            orient='index'
        ).fillna(0)
        
        combined_absorbance_df = pd.concat(
            [peptide_absorbance_df, function_absorbance_df],
            axis=1
        ).T
        
        # Create combined DataFrame with formatted values
        combined_df = pd.DataFrame(
            index=combined_absorbance_df.index,
            columns=combined_absorbance_df.columns
        )
        
        for col in combined_absorbance_df.columns:
            for idx in combined_absorbance_df.index:
                abundance = combined_absorbance_df.loc[idx, col]
                count = (combined_count_df.loc['Counts of peptides', col]
                        if idx == 'Summed Absorbance'
                        else combined_count_df.loc[idx, col])
                combined_df.loc[idx, col] = "-" if (abundance == 0 and count == 0) else f"{abundance:.2e} ({round(count)})"
        
        combined_df.rename(index={'Summed Absorbance': 'Total'}, inplace=True)
        
        return combined_df, combined_count_df, combined_absorbance_df
        
    def _on_download_plot_click(self, b):
        """Handle plot download based on selected plot type"""
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        
        with self.export_output:
            self.export_output.clear_output(wait=True)
            plot_type = self.plot_type.value
            
            if plot_type in ['Stacked Bar Plots', 'Grouped Bar Plots']:
                # Get current figure based on plot type and bar plot type
                if plot_type == 'Stacked Bar Plots':
                    fig = self.current_fig_abs  # We're only using fig_abs now for all types
                else:
                    fig = self.current_fig_abs  # Similarly for grouped plots
                    
                if fig is not None:
                    plot_type_str = plot_type.lower().replace(' ', '_')
                    bar_type_str = self.bar_plot_type.value.lower().replace(' ', '_')
                    filename = f"bioactive_{plot_type_str}_{bar_type_str}_{timestamp}.html"
                    
                    display(HTML(f'''
                        <div id="download_container_{timestamp}">
                            <a id="download_link_{timestamp}" 
                               href="data:text/html;charset=utf-8;base64,{base64.b64encode(fig.to_html().encode()).decode()}" 
                               download="{filename}"
                               style="display: none;"></a>
                            <script>
                                setTimeout(function() {{
                                    document.getElementById('download_link_{timestamp}').click();
                                }}, 100);
                            </script>
                        </div>
                    '''))
                else:
                    display(HTML("<div style='color: red; padding: 10px;'>No plot available to download.</div>"))
                    
            elif plot_type == 'Pie Charts':
                if not self.current_pie_charts:
                    display(HTML("<div style='color: red; padding: 10px;'>No pie charts available to download.</div>"))
                    return
                    
                selected_groups = list(self.group_select.value)
                
                # Create HTML wrapper that combines all pie charts
                combined_html = '''
                <html>
                <head>
                    <title>Bioactive Pie Charts</title>
                    <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
                </head>
                <body>
                '''
                
                # Add each pie chart
                for i, fig in enumerate(self.current_pie_charts):
                    group_name = selected_groups[i]
                    div_id = f'chart_{i}'
                    combined_html += f'<div id="{div_id}" style="width: 1600px; height: 700px;"></div>\n'
                    combined_html += f'<script>{fig.to_json()}</script>\n'
                    combined_html += f'''
                    <script>
                        Plotly.newPlot("{div_id}", {fig.to_json()});
                    </script>
                    '''
                
                combined_html += '</body></html>'
                
                display(HTML(f'''
                    <div id="download_container_{timestamp}">
                        <a id="download_link_{timestamp}" 
                           href="data:text/html;charset=utf-8;base64,{base64.b64encode(combined_html.encode()).decode()}" 
                           download="bioactive_pie_charts_{timestamp}.html"
                           style="display: none;"></a>
                        <script>
                            setTimeout(function() {{
                                document.getElementById('download_link_{timestamp}').click();
                            }}, 100);
                        </script>
                    </div>
                '''))

    """
    def _on_download_all_png_click(self, b):
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        
        with self.export_output:
            self.export_output.clear_output(wait=True)
            
            # Create ZIP file in memory
            zip_buffer = io.BytesIO()
            with zipfile.ZipFile(zip_buffer, 'w', zipfile.ZIP_DEFLATED) as zip_file:
                selected_groups = list(self.group_select.value)
                plot_type = self.plot_type.value
                
                # Add current bar plots based on plot type
                if plot_type in ['Stacked Bar Plots', 'Grouped Bar Plots']:
                    plot_type_str = plot_type.lower().replace(' ', '_')
                    
                    if self.current_fig_abs is not None:
                        img_buffer = io.BytesIO()
                        self.current_fig_abs.write_image(img_buffer, format='png', width=1000, height=600)
                        zip_file.writestr(f'{plot_type_str}_absolute_{timestamp}.png', img_buffer.getvalue())
                    
                    if self.current_fig_rel is not None:
                        img_buffer = io.BytesIO()
                        self.current_fig_rel.write_image(img_buffer, format='png', width=1000, height=600)
                        zip_file.writestr(f'{plot_type_str}_relative_{timestamp}.png', img_buffer.getvalue())
                
                # Add pie charts if they exist
                elif plot_type == 'Pie Charts' and self.current_pie_charts:
                    for i, fig in enumerate(self.current_pie_charts):
                        group_name = selected_groups[i]
                        img_buffer = io.BytesIO()
                        fig.write_image(img_buffer, format='png', width=1600, height=700)
                        zip_file.writestr(f'pie_charts_{group_name}_{timestamp}.png', img_buffer.getvalue())
                
                # If no plots exist, show error
                if not (self.current_fig_abs or self.current_fig_rel or self.current_pie_charts):
                    display(HTML("<div style='color: red; padding: 10px;'>No plots available to download.</div>"))
                    return
            
            # Create download link for ZIP file
            zip_buffer.seek(0)
            b64_zip = base64.b64encode(zip_buffer.getvalue()).decode()
            
            display(HTML(f'''
                <div id="download_container_{timestamp}">
                    <a id="download_link_{timestamp}" 
                       href="data:application/zip;base64,{b64_zip}" 
                       download="bioactive_plots_{timestamp}.zip"
                       style="display: none;"></a>
                    <script>
                        setTimeout(function() {{
                            document.getElementById('download_link_{timestamp}').click();
                        }}, 100);
                    </script>
                </div>
            '''))"""

                    
    def _download_pie_charts(self):
        """Download pie charts as HTML files"""
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        
        with self.export_output:
            self.export_output.clear_output(wait=True)
            
            if not self.current_pie_charts:
                display(HTML("<div style='color: red; padding: 10px;'>No pie charts available to download.</div>"))
                return
                
            # Get selected groups
            selected_groups = list(self.group_select.value)
            
            # Download each pie chart
            for i, fig in enumerate(self.current_pie_charts):
                group_name = selected_groups[i]
                filename = f'bioactive_pie_charts_{group_name}_{timestamp}.html'
                self._download_html_plot(fig, filename)
            
    def _download_html_plot(self, fig, filename):
        """Helper method to download a single HTML plot"""
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        display(HTML(f'''
            <div id="download_container_{timestamp}">
                <a id="download_link_{timestamp}" 
                   href="data:text/html;charset=utf-8;base64,{base64.b64encode(fig.to_html().encode()).decode()}" 
                   download="{filename}"
                   style="display: none;"></a>
                <script>
                    document.getElementById('download_link_{timestamp}').click();
                </script>
            </div>
        '''))
        
    def _on_export_button_click(self, b):
        """Handle data export"""
        try:
            # Process the data
            results = self._process_export_data()
            if results is None:
                with self.export_output:
                    self.export_output.clear_output(wait=True)
                    display(HTML('<div style="color: red; padding: 10px;">No bioactive data to export.</div>'))
                return
                
            combined_df, combined_count_df, combined_absorbance_df = results
            
            # Create Excel file in memory
            output = io.BytesIO()
            with pd.ExcelWriter(output, engine='openpyxl') as writer:
                combined_df.to_excel(writer, sheet_name='combined', index=True)
                combined_count_df.to_excel(writer, sheet_name='count', index=True)
                combined_absorbance_df.to_excel(writer, sheet_name='absorbance', index=True)
            
            # Get the value of the BytesIO buffer
            excel_data = output.getvalue()
            
            # Generate filename
            timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
            filename = f"Processed_mbpdb_results_{timestamp}.xlsx"
            
            # Create download link
            with self.export_output:
                self.export_output.clear_output(wait=True)
                display(HTML(f'''
                    <div id="export_container_{timestamp}">
                        <a id="export_link_{timestamp}" 
                           href="data:application/vnd.openxmlformats-officedocument.spreadsheetml.sheet;base64,{base64.b64encode(excel_data).decode()}" 
                           download="{filename}"
                           style="display: none;"></a>
                    </div>
                    <script>
                        document.getElementById('export_link_{timestamp}').click();
                    </script>
                '''))
                
        except Exception as e:
            with self.export_output:
                self.export_output.clear_output(wait=True)
                display(HTML(f'<div style="color: red; padding: 10px;">Error exporting data: {str(e)}</div>'))
    def display(self):
        """Display the bioactive peptide analysis interface"""
        display(self.widget_box)

In [4]:
# Initialize the interface
data_transformer = DataTransformation()
data_transformer.setup_data_loading_ui()

# Create bioactive plotter
bioactive_plotter = BioactivePlotter(data_transformer)
bioactive_plotter.display()

Output(layout=Layout(margin='0 0 0 20px', width='300px'))